# Grad CAM을 이용한 딥러닝 모형 해석
- http://freesearch.pe.kr/archives/4685

In [3]:
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
from collections import Counter
import operator
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
mecab = Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
tbl = pd.read_csv("ratings_train.txt",sep='\t')

In [ ]:
tbl.iloc[:10]

In [ ]:
keywords = [mecab.morphs(str(i).strip()) for i in tbl['document']]

In [ ]:
np.median([len(k) for k in keywords]), len(keywords), tbl.shape

In [ ]:
keyword_cnt = Counter([i for item in keywords for i in item])


In [ ]:
#간단하게 진행하기 위해 가장 빈도수가 많은 상위 5000개의 키워드만 사용한다. 
keyword_clip = sorted(keyword_cnt.items(), key=operator.itemgetter(1))[-5000:]